In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
df = pd.read_csv('../data/Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [11]:
X = df.drop('sales', axis = 1)
y = df['sales']
print(f'X shape: {X.shape}, y shape: {y.shape}')

X shape: (200, 3), y shape: (200,)


In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [14]:
poly_converter = PolynomialFeatures(degree = 3, include_bias = False)
poly_features = poly_converter.fit_transform(X)
print(f'{poly_features.shape}, {X.shape}')

(200, 19), (200, 3)


In [15]:
print(poly_features[0])

[2.30100000e+02 3.78000000e+01 6.92000000e+01 5.29460100e+04
 8.69778000e+03 1.59229200e+04 1.42884000e+03 2.61576000e+03
 4.78864000e+03 1.21828769e+07 2.00135918e+06 3.66386389e+06
 3.28776084e+05 6.01886376e+05 1.10186606e+06 5.40101520e+04
 9.88757280e+04 1.81010592e+05 3.31373888e+05]


In [16]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)
print(f'{X_train.shape}, {X_test.shape}, {y_train.shape}, {y_test.shape}')

(140, 19), (60, 19), (140,), (60,)


In [17]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
print(X_train[0])

[ 0.49300171 -0.33994238  1.61586707  0.28407363 -0.02568776  1.49677566
 -0.59023161  0.41659155  1.6137853   0.08057172 -0.05392229  1.01524393
 -0.36986163  0.52457967  1.48737034 -0.66096022 -0.16360242  0.54694754
  1.37075536]


In [20]:
from sklearn.linear_model import Ridge

In [21]:
ridge_model = Ridge(alpha = 10)
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [22]:
preds = ridge_model.predict(X_test)
print(preds)

[13.7469803  19.17764    11.8623393  16.25574551  8.92891155  8.04595366
 20.13320851 16.94847676 10.29219174 19.78365691 10.31758361 13.44077317
 12.24228062 23.23864236 19.12656362  9.748809   12.84361521  9.14248129
  8.82504966 21.46989971  8.25222201 18.8110776  28.39227333 25.06911066
  9.11912241 11.8071711  20.85199462  9.31962495 12.25122303  9.07811198
  9.25275556 19.87154568 10.55955497  7.86818084 17.67220132  9.31442423
 10.1539168   9.47802944  8.63337302 10.76209179 11.63848891 10.38883411
 10.90270005  8.40019284 11.35336224 10.3514086   8.00082147 15.27139854
 12.94761469 22.49762978 10.80165407 13.01337743 14.82204895 11.37354043
 11.69628953  7.20576885 25.53512232 10.41926545 17.38798022 15.10158819]


In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
Mae = mean_absolute_error(y_test, preds)
Mse = mean_squared_error(y_test, preds)
print(f'Mae: {Mae}, Mse: {Mse}')

Mae: 0.5774404204714162, Mse: 0.8003783071528354


In [24]:
from sklearn.linear_model import RidgeCV
ridge_cv_model = RidgeCV( alphas= [0.1, 1.0, 10.0], scoring='neg_mean_absolute_error')

In [25]:
ridge_cv_model.fit(X_train, y_train)
preds = ridge_cv_model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)
print(mae, mse)

0.4273774884290877 0.3820129881478915


In [26]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [27]:
ridge_cv_model.alpha_

0.1

In [31]:
from sklearn.linear_model import LassoCV
lasso_model = LassoCV(eps = 0.1, n_alphas = 100, cv = 5)
lasso_model.fit(X_train, y_train)
preds = lasso_model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)
print(mae, mse)

0.6541723161252868 1.2787088713079886


In [32]:
lasso_model.alpha_

0.4943070909225831

In [33]:
lasso_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [34]:
from sklearn.linear_model import ElasticNetCV
elastic_model = ElasticNetCV(l1_ratio= [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], eps = 0.1, n_alphas = 100, cv = 5)
elastic_model.fit(X_train, y_train)
preds = elastic_model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)
print(mae, mse)

0.6541723161252868 1.2787088713079886


In [37]:
print(elastic_model.alpha_)
print(elastic_model.coef_)
print(elastic_model.l1_ratio_)

0.4943070909225831
[1.002651   0.         0.         0.         3.79745279 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
1.0
